In [1]:
var fs = require('fs').promises

In [2]:
var raw
var data
fs.readFile('./data-12.txt','utf8').then((file) => {
  raw = file
  data = raw.split('\r\n').map(x=>x.split(''))
}).catch((err) => {
  console.log(err)
});


Promise { <pending> }

In [3]:
var start
var end
for (let i = 0; i < data.length; i++) {
  let e = data[i].indexOf('S');
  if (e>-1){
    start = [i,e]
  }
  e = data[i].indexOf('E');
  if (e>-1){
    end = [i,e]
  }
  if (start && end) break
}
[start,end]

[ [ 20, 0 ], [ 20, 145 ] ]

In [4]:
class NODE{
  constructor(x,y,value){
    this.x = x
    this.y = y
    this.value = value
    this.point = value.charCodeAt() - 97
    this.type = (()=>{
      if (value == 'S') {
        this.point = 0
        return 'start'
      }
      if (value == 'E') {
        this.point = 122-97
        return 'end'
      }
      return 'node'
    })()
    this.h = Math.abs(end[1] - x) + Math.abs(end[0] - y)
    // this.g = Math.abs(start[1] - x) + Math.abs(start[0] - y)
    this.prev
    this.next
    this.visited = false

    this.vecinos = []
    this.path = false
    this.cost = 0
    this.total = 10000
    this.prev_axi = [0,0]
    this.key = y*1000 + x
  }
  g(vecino){
    let result = vecino.point - this.point 
    result = result > 1 ? 10000 : Math.abs(result)
    let axi = Math.abs(vecino.prev_axi[0] - this.prev_axi[0]) + Math.abs(vecino.prev_axi[1] - this.prev_axi[1]) 
    result = result + this.cost + 1 + axi
    return result
  }
  enable(){
    return this.vecinos.every(x => !x.visited)
  }
}

In [5]:
function aStar(){
  const xMax = data[0].length
  const yMax = data.length

  let maps = Array(yMax).fill(null).map(()=> Array(xMax).fill(null))

  const directions = [[1,0],[0,1],[0,-1],[-1,0]]

  maps[start[0]][start[1]] = new NODE(start[1],start[0],'S')
  maps[end[0]][end[1]] = new NODE(end[1],end[0],'E')
  
  let pos = maps[start[0]][start[1]]

  let nodes = {[start[0]*1000+start[1]]: pos}
  
  let empty = false;
  while(true){
    for(let [x2,y2] of directions){
      let x = pos.x + x2
      let y = pos.y + y2
      
      if (x < 0 || y < 0 || x >= xMax || y >= yMax)  continue

      if(!maps[y][x]){
        maps[y][x] = new NODE(x,y,data[y][x])
      }
      const next = maps[y][x] 
      
      pos.vecinos.push(next)
      if (next.visited) continue
      
      next.prev_axi = [Math.abs(x), Math.abs(y)]
      
      const f = pos.g(next) + next.h
      
      const k = y*1000+x
      
      if (f >= 10000) continue
      next.cost = pos.g(next)
      
      if(nodes.hasOwnProperty(k)){
        const old = nodes[k].f

        if( old > f){
          next.prev = pos
          nodes[k].f = f
          nodes[k].prev = pos
        }
      }else{ 
        next.prev = pos
        nodes[k] = {
          f,
          node: next,
          prev: pos
        }
      }
    }
    pos.visited = true

    delete nodes[pos.key]

    const arr = Object.entries(nodes)

    if (arr.length == 0){
      empty = true
      break
    }
    pos = arr.reduce((prev, curr)=>{
      return prev[1].f < curr[1].f ? prev : curr
      
    })[1].node
    if (pos.type == 'end') break
    
  }
  let step = 0

  if (empty) return null
  
  while (true) {
    pos.path = true
    if (pos.type == 'start') break
    step++
    pos = pos.prev
  }
  let text = '<span style="font-family:\'Lucida Console\', monospace">'
  for (const [i,y] of maps.entries()){
    for (const [j,x] of y.entries()){
      if(x){
        if(x.path){
          text = text + '<span style="color:red;">' + data[i][j] + '</span>'
          continue
        }
      }
      text += data[i][j]
    }
    text += '\n'
  }
  text += '</span>'
  fs.writeFile('./result.html', text)
  return step
}

aStar()

517

# Parte 2

In [6]:
var a = []

for (let i = 0; i < data.length; i++){
  a.push([i,0])
}
a

[
  [ 0, 0 ],  [ 1, 0 ],  [ 2, 0 ],  [ 3, 0 ],
  [ 4, 0 ],  [ 5, 0 ],  [ 6, 0 ],  [ 7, 0 ],
  [ 8, 0 ],  [ 9, 0 ],  [ 10, 0 ], [ 11, 0 ],
  [ 12, 0 ], [ 13, 0 ], [ 14, 0 ], [ 15, 0 ],
  [ 16, 0 ], [ 17, 0 ], [ 18, 0 ], [ 19, 0 ],
  [ 20, 0 ], [ 21, 0 ], [ 22, 0 ], [ 23, 0 ],
  [ 24, 0 ], [ 25, 0 ], [ 26, 0 ], [ 27, 0 ],
  [ 28, 0 ], [ 29, 0 ], [ 30, 0 ], [ 31, 0 ],
  [ 32, 0 ], [ 33, 0 ], [ 34, 0 ], [ 35, 0 ],
  [ 36, 0 ], [ 37, 0 ], [ 38, 0 ], [ 39, 0 ],
  [ 40, 0 ]
]

In [7]:
a.map(n=>{
  start = n
  // console.log(start)
  return aStar()
}).reduce((a,b)=> a<b? a : b)

512